In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy import stats
import krippendorff
import matplotlib.pyplot as plt

In [7]:
path = "experiments/test_local"
results_df = pd.read_parquet(path+"/results.parquet")

print(len(results_df))
results_df["llm_decision"].value_counts()
print(len(results_df["combination_id"].unique()))
# results_df.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)
results_df




9720
324


,experiment_id,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,reasoning_length,processing_time,total_tokens,tokens_per_second,final_prompt_type_category,final_prompt_type_value,think_mode_category,think_mode_value,predicted_target
0,test_local,0,0,8723,Check-Out,Check-Out,Check-Out,0,2.0428,1801,881.6413,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
1,test_local,0,1,10233,Check-Out,Check-Out,Check-Out,0,1.4796,1779,1202.3290,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
2,test_local,0,2,52028,Cancellation,Cancellation,Check-Out,0,1.2442,1747,1404.1469,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
3,test_local,0,3,16948,Check-Out,Cancellation,Check-Out,0,1.2452,1727,1386.9296,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
4,test_local,0,4,16376,Check-Out,Check-Out,Check-Out,0,1.2463,1738,1394.4747,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9715,test_local,323,25,84583,Check-Out,Check-Out,Check-Out,2737,21.5156,2350,109.2231,ai_neutral_rational,The dataset above contains hotel booking detai...,cot,Lay out a strategy how to go at this task. The...,None
9716,test_local,323,26,96373,Check-Out,Cancellation,Cancellation,2768,23.5077,2449,104.1786,ai_neutral_rational,The dataset above contains hotel booking detai...,cot,Lay out a strategy how to go at this task. The...,None
9717,test_local,323,27,99102,Check-Out,Check-Out,Check-Out,2283,18.3214,2269,123.8446,ai_neutral_rational,The dataset above contains hotel booking detai...,cot,Lay out a strategy how to go at this task. The...,None
9718,test_local,323,28,106685,Check-Out,Check-Out,Check-Out,3167,30.1099,2607,86.5828,ai_neutral_rational,The dataset above contains hotel booking detai...,cot,Lay out a strategy how to go at this task. The...,None


In [3]:
results_df["processing_time"].mean()

np.float64(10.674307202655102)

In [16]:
results_df["final_prompt_type_category"].unique()

array(['ai_trust_emotional', 'ai_distrust_emotional',
       'ai_neutral_emotional', 'ai_trust_rational',
       'ai_distrust_rational', 'ai_neutral_rational'], dtype=object)

In [12]:
results_df.fillna("Unchanged", inplace=True)

In [10]:
path = "experiments/done/prompt_variations_exclusion_pdp-histogram"
results_df = pd.read_parquet(path+"/results.parquet")

path = "experiments/done/prompt_variations_exclusion_shap-lime"
results_df2 = pd.read_parquet(path+"/results.parquet")
results_df2.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)

print(results_df2["final_prompt_type_category"].unique())
results_df2["combination_id"] = results_df2["combination_id"].apply(lambda x: x+100)
results_df2["xai_exclusion_pdp"] = False
results_df2["xai_exclusion_histogram"] = False


results_df["xai_exclusion_shap"] =False
results_df["xai_exclusion_lime_explanation"] = False
results_df = pd.concat([results_df, results_df2], ignore_index=True)
results_df

from rixeval.analysis import ResultAnalyzer
results_df_temp = results_df
analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
print("FN: ", summary["false_negative_rate"].mean())
print("FP: ",summary["false_positive_rate"].mean())

['direct_answers' 'cot' 'emotional_tone' 'ai_distrust']
FN:  0.1407894736842105
FP:  0.23157894736842105


In [16]:
# results_df["method_order"] = results_df["method_order"].apply(lambda x: str(x)[1:-1])
# results_df

FN:  0.17194444444444443
FP:  0.22472222222222218


In [8]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
summary


columns = summary.columns
for column in columns:
    if column in ['n_datapoints', 'percent_correct', 'percent_agree', 'corr_correct',
       'corr_agree', 'false_positive_rate', 'false_negative_rate',"think_mode_value","avg_processing_time",
                  "final_prompt_type_value"]:
        continue

    marginal = column#"final_prompt_type_category"
    comparison = analyzer.compare_parameters(marginal,True)
    comparison = comparison.round(4).drop(["n_combinations"], axis=1)
    comparison

    to_drop = ["final_prompt_type_value", "think_mode_value", "n_datapoints", "avg_processing_time"]
    for drop_col in to_drop:
        if drop_col in comparison.columns:
            comparison.drop(drop_col, axis=1, inplace=True)
    df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
    df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
                             "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
                             "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
                             "avg_false_positive_rate": "$\\overline{FP}$",
                             "final_prompt_type_category":"Prompt category"}, inplace=True)
    table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
                            column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
    table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
    print(table_str)
    print()

#\hspace*{-1.5cm}


# df_t = df_clean.T
# df_t = df_t.set_axis(df_t.iloc[0], axis=1).iloc[1:]
# df_t.columns = df_t.columns.str.replace('_', ' ')
#
# print(df_t.to_latex(escape=True, float_format="%.2f", caption=marginal.replace("_", " "),
#                                                              column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c"))



\begin{table}
\caption{final prompt type category}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
Prompt category & $\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{FN}$ \\
\midrule
ai trust emotional & 0.02 & 56.60+-14.74 & 57.84+-24.56 & 0.07+-0.22 & 0.10+-0.49 & 0.22+-0.13 & 0.21+-0.25 \\
ai distrust emotional & 0.06 & 52.04+-13.73 & 50.31+-19.91 & 0.05+-0.23 & -0.01+-0.42 & 0.16+-0.11 & 0.32+-0.21 \\
ai neutral emotional & 0.01 & 54.94+-15.14 & 54.44+-25.96 & 0.01+-0.24 & 0.02+-0.52 & 0.24+-0.12 & 0.21+-0.24 \\
ai trust rational & 0.03 & 55.99+-15.64 & 56.73+-22.66 & 0.03+-0.25 & 0.08+-0.45 & 0.25+-0.12 & 0.19+-0.23 \\
ai distrust rational & 0.04 & 57.35+-11.64 & 56.48+-16.00 & 0.03+-0.19 & 0.04+-0.33 & 0.27+-0.11 & 0.16+-0.19 \\
ai neutral rational & 0.00 & 54.51+-16.06 & 53.40+-23.58 & 

In [13]:
mask = results_df['predicted_target'] != 'Unchanged'
results_df.loc[mask, 'predicted_outcome'] = results_df.loc[mask, 'predicted_target']

In [14]:
results_df

,experiment_id,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,reasoning_length,processing_time,total_tokens,tokens_per_second,final_prompt_type_category,final_prompt_type_value,think_mode_category,think_mode_value,predicted_target
0,result_manipulation_new_prompts,0,0,8723,Check-Out,Check-Out,Check-Out,0,0.4543,1659,3652.0965,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
1,result_manipulation_new_prompts,0,1,10233,Check-Out,Check-Out,Check-Out,0,0.2755,1637,5941.6654,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
2,result_manipulation_new_prompts,0,2,52028,Cancellation,Check-Out,Check-Out,0,0.2676,1616,6039.0025,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
3,result_manipulation_new_prompts,0,3,16948,Check-Out,Check-Out,Check-Out,0,0.2839,1601,5639.7402,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
4,result_manipulation_new_prompts,0,4,16376,Check-Out,Check-Out,Check-Out,0,0.2703,1599,5916.1783,ai_trust_emotional,Take a deep breath and feel confident in this ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Check-Out
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,result_manipulation_new_prompts,575,25,84583,Check-Out,Check-Out,Check-Out,2663,6.2843,2210,351.6683,ai_neutral_rational,Analyze the provided data and XAI methods from...,cot,Analyze this step-by-step:\n\nThen provide you...,Unchanged
17276,result_manipulation_new_prompts,575,26,96373,Check-Out,Cancellation,Cancellation,2992,7.0484,2274,322.6286,ai_neutral_rational,Analyze the provided data and XAI methods from...,cot,Analyze this step-by-step:\n\nThen provide you...,Unchanged
17277,result_manipulation_new_prompts,575,27,99102,Check-Out,Check-Out,Check-Out,2718,4.9095,2215,451.1659,ai_neutral_rational,Analyze the provided data and XAI methods from...,cot,Analyze this step-by-step:\n\nThen provide you...,Unchanged
17278,result_manipulation_new_prompts,575,28,106685,Check-Out,Check-Out,Check-Out,2934,7.5650,2274,300.5950,ai_neutral_rational,Analyze the provided data and XAI methods from...,cot,Analyze this step-by-step:\n\nThen provide you...,Unchanged


In [17]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
summary
comparison = analyzer.compare_parameters("predicted_target",True)
comparison = comparison.round(4).drop(["n_combinations"], axis=1)
comparison


# df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
# df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
#                          "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
#                          "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
#                          "avg_false_positive_rate": "$\\overline{FP}$",
#                          "final_prompt_type_category":"Prompt category"}, inplace=True)
# table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
#                         column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
# table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
# print(table_str)


/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: Runtime

,predicted_target,krippendorff_alpha,avg_percent_correct,avg_percent_agree,avg_corr_correct,avg_corr_agree,avg_false_positive_rate,avg_false_negative_rate
0,Check-Out,0.3215,61.25+-6.16,84.25+-14.28,0.05+-0.17,0.00+-0.00,0.30+-0.07,0.09+-0.09
1,Cancellation,0.1931,52.50+-13.41,48.83+-28.33,0.05+-0.21,nan+-nan,0.18+-0.09,0.30+-0.20
2,Unchanged,0.5110,67.25+-6.58,84.08+-13.83,0.28+-0.13,0.68+-0.27,0.20+-0.07,0.13+-0.09


In [3]:
import pandas as pd

data = {
    'XAI Method': ['SHAP', 'SHAP', 'PDP', 'PDP', 'LIME', 'LIME', 'Counterfactual', 'Counterfactual', 'Histograms', 'Histograms', 'All'],
    'Exclusion': ['No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes'],
    'α': [0.56, 0.68, 0.65, 0.59, 0.66, 0.59, 0.63, 0.62, 0.61, 0.62, 0.51],
    'Avg. % correct': [67.05, 67.64, 67.66, 67.04, 67.81, 66.88, 66.29, 68.40, 67.35, 67.34, 67.67],
    'correct_std': [5.79, 4.98, 4.65, 6.06, 5.00, 5.75, 5.64, 4.95, 5.11, 5.70, 7.17],
    'Avg. % agree': [85.51, 90.48, 88.85, 87.14, 89.44, 86.56, 87.46, 88.54, 87.95, 88.05, 84.50],
    'agree_std': [12.48, 10.28, 10.35, 12.86, 10.75, 12.41, 11.38, 12.00, 11.57, 11.84, 14.03],
    'r_correct': [0.27, 0.30, 0.28, 0.28, 0.30, 0.27, 0.25, 0.31, 0.28, 0.28, 0.28],
    'r_correct_std': [0.15, 0.11, 0.13, 0.13, 0.12, 0.14, 0.14, 0.11, 0.12, 0.14, 0.15],
    'r_agree': [0.70, 0.81, 0.77, 0.74, 0.79, 0.73, 0.74, 0.77, 0.76, 0.76, 0.68],
    'r_agree_std': [0.26, 0.20, 0.22, 0.26, 0.21, 0.26, 0.24, 0.24, 0.24, 0.24, 0.30],
    'FP_mean': [0.20, 0.18, 0.19, 0.19, 0.18, 0.19, 0.20, 0.18, 0.19, 0.19, 0.20],
    'FP_std': [0.06, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.07],
    'FN_mean': [0.13, 0.15, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.12],
    'FN_std': [0.06, 0.06, 0.04, 0.07, 0.06, 0.06, 0.05, 0.07, 0.06, 0.06, 0.09]
}

df = pd.DataFrame(data)
df

,XAI Method,Exclusion,α,Avg. % correct,correct_std,Avg. % agree,agree_std,r_correct,r_correct_std,r_agree,r_agree_std,FP_mean,FP_std,FN_mean,FN_std
0,SHAP,No,0.56,67.05,5.79,85.51,12.48,0.27,0.15,0.70,0.26,0.20,0.06,0.13,0.06
1,SHAP,Yes,0.68,67.64,4.98,90.48,10.28,0.30,0.11,0.81,0.20,0.18,0.05,0.15,0.06
2,PDP,No,0.65,67.66,4.65,88.85,10.35,0.28,0.13,0.77,0.22,0.19,0.06,0.14,0.04
3,PDP,Yes,0.59,67.04,6.06,87.14,12.86,0.28,0.13,0.74,0.26,0.19,0.06,0.14,0.07
4,LIME,No,0.66,67.81,5.00,89.44,10.75,0.30,0.12,0.79,0.21,0.18,0.06,0.14,0.06
5,LIME,Yes,0.59,66.88,5.75,86.56,12.41,0.27,0.14,0.73,0.26,0.19,0.06,0.14,0.06
6,Counterfactual,No,0.63,66.29,5.64,87.46,11.38,0.25,0.14,0.74,0.24,0.20,0.06,0.14,0.05
7,Counterfactual,Yes,0.62,68.40,4.95,88.54,12.00,0.31,0.11,0.77,0.24,0.18,0.06,0.14,0.07
8,Histograms,No,0.61,67.35,5.11,87.95,11.57,0.28,0.12,0.76,0.24,0.19,0.06,0.14,0.06
9,Histograms,Yes,0.62,67.34,5.70,88.05,11.84,0.28,0.14,0.76,0.24,0.19,0.06,0.14,0.06


In [5]:
df_no = df[df['Exclusion'] == 'No']
df_no["Krippendorffs Alpha"].mean()

np.float64(0.6224000000000001)